In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [549]:
# 어떤 행렬을 정의한다.

x = np.matrix([[1,5,3,4],[2,3,6,2],[3,1,7,8]])

# 5. LU분해

## 1) 기본 개념
### (1) 기본 개념은, 각각 소블럭의 1행과 1열을 외적하여 행렬로 전개한 값을, 원래 행렬에서 지속적으로 빼서
### (2) a11 성분으로 나누어 1행과 1열을 원래 행렬의 값과 동일하게 만들어 준 후에
### (3) 1행과 1열을 0으로 만들어주면서 지속적으로 축소해준다.

In [541]:
normalizer = 1/(x[0,0])
temp_l = normalizer * x[0].T
temp_u = x.T[0]
l = normalizer * x[0].T
u = x.T[0]
temp = np.matmul(temp_l,temp_u).T
x = x - temp

In [542]:
normalizer = 1/x[1,1]
temp_l = normalizer * x[1].T
temp_u = x.T[1]
l = np.hstack([l,temp_l])
u = np.vstack([u,temp_u])
temp = np.matmul(temp_l,temp_u).T
x = x - temp

In [543]:
normalizer = 1/x[2,2]
temp_l = normalizer * x[2].T
temp_u = x.T[2]
l = np.hstack([l,temp_l])
u = np.vstack([u,temp_u])
temp = np.matmul(temp_l,temp_u).T
x = x - temp

In [567]:
def LUdecomp(x) :
    if type(x) != type(np.matrix([0])):
        raise ValueError 
    s = np.min(x.shape)
    
    normalizer = 1/(x[0,0])
    l = normalizer * x[0].T
    u = x.T[0]
    temp = np.matmul(l,u).T
    x = x - temp
    
    for i in range(1,s):
        normalizer = 1/x[i,i]
        temp_l = normalizer * x[i].T
        temp_u = x.T[i]
        l = np.hstack([l,temp_l])
        u = np.vstack([u,temp_u])
        temp = np.matmul(temp_l,temp_u).T
        x = x - temp
    return (l,u)

In [569]:
L,U = LUdecomp(x)